In [1]:
# airflow/scripts/extract_coingecko_data.py
import requests
import json
import pandas as pd
from datetime import datetime
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook

[2025-11-29T00:36:01.997-0800] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.filesystem.FSHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work
[2025-11-29T00:36:02.001-0800] {providers_manager.py:953} INFO - The hook_class 'airflow.providers.standard.hooks.package_index.PackageIndexHook' is not fully initialized (UI widgets will be missing), because the 'flask_appbuilder' package is not installed, however it is not required for Airflow components to work


In [2]:
#Fetches a list of all supported coins from CoinGecko API.
# get_coin_list():
url = "https://api.coingecko.com/api/v3/coins/list"
response = requests.get(url, timeout=10)
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
crypto_list = response.json()
crypto_list
    #     return response.json()
    # except requests.exceptions.RequestException as e:
    #     print(f"Error fetching coin list: {e}")
    #     return None

[{'id': '_', 'symbol': 'gib', 'name': '༼ つ ◕_◕ ༽つ'},
 {'id': '000-capital', 'symbol': '000', 'name': '000 Capital'},
 {'id': '01111010011110000110001001110100-token',
  'symbol': '01111010011110000110001001110100',
  'name': '01111010011110000110001001110100'},
 {'id': '01-token', 'symbol': '01', 'name': '01'},
 {'id': '0chain', 'symbol': 'zcn', 'name': 'Zus'},
 {'id': '0vix-protocol', 'symbol': 'vix', 'name': '0VIX Protocol'},
 {'id': '0x', 'symbol': 'zrx', 'name': '0x Protocol'},
 {'id': '0x0-ai-ai-smart-contract',
  'symbol': '0x0',
  'name': '0x0.ai: AI Smart Contract'},
 {'id': '0x678-landwolf-1933', 'symbol': 'wolf', 'name': 'Landwolf'},
 {'id': '0xgasless-2', 'symbol': '0xgas', 'name': '0xGasless'},
 {'id': '0xgen', 'symbol': 'xgn', 'name': '0xGen'},
 {'id': '0x-leverage', 'symbol': 'oxl', 'name': '0x Leverage'},
 {'id': '0xlsd', 'symbol': '0xlsd', 'name': '0xLSD'},
 {'id': '0xmonk', 'symbol': 'monk', 'name': '0xMonk by Virtuals'},
 {'id': '0x-protocol-avalanche-bridged-zrx-e',


In [3]:
len(crypto_list)

19356

In [4]:
coin_id_list = [c.get('id') for c in crypto_list]
coin_id_list

['_',
 '000-capital',
 '01111010011110000110001001110100-token',
 '01-token',
 '0chain',
 '0vix-protocol',
 '0x',
 '0x0-ai-ai-smart-contract',
 '0x678-landwolf-1933',
 '0xgasless-2',
 '0xgen',
 '0x-leverage',
 '0xlsd',
 '0xmonk',
 '0x-protocol-avalanche-bridged-zrx-e',
 '0xshadow',
 '0xsim-by-virtuals',
 '0xy',
 '-10',
 '1000bonk',
 '1000btt',
 '1000cat',
 '1000chems',
 '1000mog',
 '1000rats',
 '1000sats-ordinals',
 '1000shib',
 '1000x-by-virtuals',
 '100-token',
 '100xdarren',
 '10-figs',
 '-11',
 '11am',
 '1984-token',
 '1art',
 '1-coin-can-change-your-life',
 '1-community-can-change-your-life',
 '1dev',
 '1-dog-can-change-your-life',
 '1-dollar-sol-coin',
 '1guy',
 '1hive-water',
 '1hub-ai',
 '1inch',
 '1inch-yvault',
 '1intro',
 '1mbabydoge',
 '1mdc',
 '1million-nfts',
 '1move-token',
 '1-narrative-can-change-your-life',
 '1-one',
 '1-percent',
 '1rus-btc25',
 '1rus-dao',
 '1-squirrel',
 '1-token',
 '2004-pepe',
 '2025-token',
 '2077-code',
 '2080',
 '21million',
 '23-turtles',
 '2

In [5]:
url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
vs_currency = 'usd'
params = {
'vs_currency': vs_currency,
'days': 365,
'interval': 'daily' # Or 'hourly', 'minutely' depending on your needs
}
response = requests.get(url, params=params, timeout=30)
response.raise_for_status()
response.json()

{'prices': [[1732924800000, 97453.2473451042],
  [1733011200000, 96513.14234698225],
  [1733097600000, 97311.70719084324],
  [1733184000000, 95833.1362300365],
  [1733270400000, 96031.63097759831],
  [1733356800000, 98881.4694561823],
  [1733443200000, 97201.50036407741],
  [1733529600000, 99973.85150659826],
  [1733616000000, 99781.82999198174],
  [1733702400000, 101235.3717025316],
  [1733788800000, 97353.94700820887],
  [1733875200000, 96649.71446771531],
  [1733961600000, 101123.61811033697],
  [1734048000000, 100000.80836535197],
  [1734134400000, 101352.22971292294],
  [1734220800000, 101367.01064553364],
  [1734307200000, 104721.50151808978],
  [1734393600000, 106074.10723541168],
  [1734480000000, 106034.913402645],
  [1734566400000, 100355.5761481501],
  [1734652800000, 97851.35377075805],
  [1734739200000, 97691.43431692653],
  [1734825600000, 97202.82496847633],
  [1734912000000, 95094.27394862173],
  [1734998400000, 94644.91085464321],
  [1735084800000, 98695.71400782601],


In [6]:
raw_data = response.json()
raw_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

In [7]:
#Fetches market chart data (price, market cap, total volume) for a coin.
# get_market_chart(coin_id, vs_currency='usd', days='max')
coins = ['bitcoin', 'ethereum']#, 'ripple', 'litecoin', 'cardano']
vs_currency = 'usd'
all_data_frames = []

for coin_id in coins:
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
    'vs_currency': vs_currency,
    'days': 364,
    'interval': 'daily' # Or 'hourly', 'minutely' depending on your needs
    }
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    coin_data = response.json()
    
    prices = coin_data.get('prices', [])
    market_caps = coin_data.get('market_caps', [])
    total_volumes = coin_data.get('total_volumes', [])
    
    df_prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df_market_caps = pd.DataFrame(market_caps, columns=['timestamp', 'market_cap'])
    df_total_volumes = pd.DataFrame(total_volumes, columns=['timestamp', 'total_volume'])

    # Convert timestamps to datetime and then to date for daily aggregation
    df_prices['date'] = pd.to_datetime(df_prices['timestamp'], unit='ms').dt.date
    df_market_caps['date'] = pd.to_datetime(df_market_caps['timestamp'], unit='ms').dt.date
    df_total_volumes['date'] = pd.to_datetime(df_total_volumes['timestamp'], unit='ms').dt.date

    # Group by date and take the last value for simplicity or average, depending on your analysis needs
    # For daily data, the last value of the day is often used.
    df_price = df_prices.groupby('date')['price'].last().reset_index()
    df_market_caps = df_market_caps.groupby('date')['market_cap'].last().reset_index()
    df_total_volumes = df_total_volumes.groupby('date')['total_volume'].last().reset_index()

    # Merge dataframes
    df = pd.merge(df_prices, df_market_caps, on='date', how='outer')
    df = pd.merge(df, df_total_volumes, on='date', how='outer')

    df['coin_id'] = coin_id
    df['extraction_timestamp'] = datetime.utcnow()

    if not df.empty:
        all_data_frames.append(df)
        print(f"Successfully processed {len(df)} records for {coin_id}.")
    else:
        print(f"No data processed for {coin_id}.")
    final_df = pd.concat(all_data_frames, ignore_index=True)
final_df['date'] = final_df['date'].astype(str)
final_df.drop(['timestamp'],axis=1,inplace=True)
final_df['date'] = final_df['date'].astype(str)



Successfully processed 365 records for bitcoin.
Successfully processed 365 records for ethereum.


In [18]:
# final_df['extraction_timestamp'] = final_df['extraction_timestamp'].astype(str)
type(final_df['extraction_timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [15]:
import os
import snowflake.connector

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=os.environ['SNOWFLAKE_USER'],
    password=os.environ['SNOWFLAKE_PASSWORD'],
    account=os.environ['SNOWFLAKE_ACCOUNT'],
    role=os.environ["SNOWFLAKE_ROLE"],
    warehouse=os.environ["SNOWFLAKE_WAREHOUSE"],
    database=os.environ["SNOWFLAKE_DATABASE"],
    schema=os.environ["SNOWFLAKE_SCHEMA"]
)

cursor = conn.cursor()
cursor.execute("SELECT 1")
result = cursor.fetchone()
print(f"Query result: {result[0]}")

[2025-11-29T00:43:43.416-0800] {connection.py:493} INFO - Snowflake Connector for Python Version: 3.16.0, Python Version: 3.11.7, Platform: macOS-26.1-x86_64-i386-64bit
[2025-11-29T00:43:43.417-0800] {connection.py:1410} INFO - Connecting to GLOBAL Snowflake domain
Query result: 1


In [10]:
sql="""
    CREATE TABLE IF NOT EXISTS CRYPTO_DB.RAW.COINGECKO_HISTORICAL_PRICES_TEMP (
        coin_id VARCHAR,
        date DATE,
        price FLOAT,
        market_cap FLOAT,
        total_volume FLOAT,
        extraction_timestamp TIMESTAMP_LTZ
    );
"""

try:
    cursor = conn.cursor()
    # Define the CREATE TABLE statement
    create_table_sql = sql
    cursor.execute(create_table_sql)
    print("Table 'my_new_table' created successfully.")

finally:
    cursor.close()
    conn.close()

Table 'my_new_table' created successfully.


In [11]:
columns_to_load = ['coin_id', 'date', 'price', 'market_cap', 'total_volume', 'extraction_timestamp']
data_to_load = [tuple(row) for row in final_df[columns_to_load].itertuples(index=False)]
for row in final_df[columns_to_load].itertuples(index=False):
    print(row)
data_to_load

Pandas(coin_id='bitcoin', date='2024-12-01', price=96513.14234698225, market_cap=1910022814269.8855, total_volume=43580019720.720634, extraction_timestamp=Timestamp('2025-11-29 08:36:36.647061'))
Pandas(coin_id='bitcoin', date='2024-12-02', price=97311.70719084324, market_cap=1925808416477.5068, total_volume=49147687432.565895, extraction_timestamp=Timestamp('2025-11-29 08:36:36.647061'))
Pandas(coin_id='bitcoin', date='2024-12-03', price=95833.1362300365, market_cap=1896576914275.523, total_volume=101019861361.49252, extraction_timestamp=Timestamp('2025-11-29 08:36:36.647061'))
Pandas(coin_id='bitcoin', date='2024-12-04', price=96031.63097759831, market_cap=1900922167493.3643, total_volume=87935752287.975, extraction_timestamp=Timestamp('2025-11-29 08:36:36.647061'))
Pandas(coin_id='bitcoin', date='2024-12-05', price=98881.4694561823, market_cap=1957276192604.2603, total_volume=98230784298.0459, extraction_timestamp=Timestamp('2025-11-29 08:36:36.647061'))
Pandas(coin_id='bitcoin', da

[('bitcoin',
  '2024-12-01',
  96513.14234698225,
  1910022814269.8855,
  43580019720.720634,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-02',
  97311.70719084324,
  1925808416477.5068,
  49147687432.565895,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-03',
  95833.1362300365,
  1896576914275.523,
  101019861361.49252,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-04',
  96031.63097759831,
  1900922167493.3643,
  87935752287.975,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-05',
  98881.4694561823,
  1957276192604.2603,
  98230784298.0459,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-06',
  97201.50036407741,
  1920742551503.6094,
  190460293531.53802,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-07',
  99973.85150659826,
  1978389972043.7947,
  115812632020.44765,
  Timestamp('2025-11-29 08:36:36.647061')),
 ('bitcoin',
  '2024-12-08',
  99781.82999

In [16]:
table_name = "COINGECKO_HISTORICAL_PRICES_TEMP"
schema_name = "RAW"
database_name = "CRYPTO_DB"
# cursor = conn.cursor()
try:
    with conn.cursor() as cur:
        # Truncate table before inserting (for full refresh)
        cur.execute(f"TRUNCATE TABLE {database_name}.{schema_name}.{table_name}")
        # Prepare the insert statement
        placeholders = ', '.join(['%s'] * len(columns_to_load))
        column_names = ', '.join(columns_to_load)
        insert_sql = f"INSERT INTO {database_name}.{schema_name}.{table_name} ({column_names}) VALUES ({placeholders})"
        # Executed batch insert
        cur.executemany(insert_sql, data_to_load)
        conn.commit()
    print(f"Successfully loaded {len(data_to_load)} records to Snowflake.")
except Exception as e:
    print(f"Error loading data to Snowflake: {e}")
    raise # Re-raise to fail the Airflow task


Error loading data to Snowflake: 252004: 252004: Failed processing pyformat-parameters; 255001: 255001: Binding data in type (timestamp) is not supported.


ProgrammingError: 252004: 252004: Failed processing pyformat-parameters; 255001: 255001: Binding data in type (timestamp) is not supported.

In [ ]:
# from airflow.providers.snowflake.operators.snowflake import SnowflakeOperator
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook

In [ ]:
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook
snowflake_hook = SnowflakeHook(snowflake_conn_id='snowflake_default') # Ensure this matches your Airflow connection ID
table_name = "COINGECKO_HISTORICAL_PRICES"
schema_name = "RAW"
database_name = "CRYPTO_DB"

try:
    with snowflake_hook.get_conn() as conn:
        with conn.cursor() as cur:
            # Truncate table before inserting (for full refresh)
            cur.execute(f"TRUNCATE TABLE {database_name}.{schema_name}.{table_name}")
            # Prepare the insert statement
            placeholders = ', '.join(['%s'] * len(columns_to_load))
            column_names = ', '.join(columns_to_load)
            insert_sql = f"INSERT INTO {database_name}.{schema_name}.{table_name} ({column_names}) VALUES ({placeholders})"
            # Executed batch insert
            cur.executemany(insert_sql, data_to_load)
            conn.commit()
    print(f"Successfully loaded {len(data_to_load)} records to Snowflake.")
except Exception as e:
    print(f"Error loading data to Snowflake: {e}")
    raise # Re-raise to fail the Airflow task


----

In [ ]:
# airflow/scripts/extract_coingecko_data.py
import requests
import json
import pandas as pd
from datetime import datetime
from airflow.providers.snowflake.hooks.snowflake import SnowflakeHook

def get_coin_list():
    """Fetches a list of all supported coins from CoinGecko API."""
    url = "https://api.coingecko.com/api/v3/coins/list"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching coin list: {e}")
        return None

def get_market_chart(coin_id, vs_currency='usd', days='max'):
    """Fetches market chart data (price, market cap, total volume) for a coin."""
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        'vs_currency': vs_currency,
        'days': days,
        'interval': 'daily' # Or 'hourly', 'minutely' depending on your needs
    }
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching market chart for {coin_id}: {e}")
        return None

def transform_market_chart_data(data, coin_id):
    """Transforms raw CoinGecko market chart data into a flat DataFrame."""
    if not data:
        return pd.DataFrame()

    prices = data.get('prices', [])
    market_caps = data.get('market_caps', [])
    total_volumes = data.get('total_volumes', [])

    df_prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df_market_caps = pd.DataFrame(market_caps, columns=['timestamp', 'market_cap'])
    df_total_volumes = pd.DataFrame(total_volumes, columns=['timestamp', 'total_volume'])

    # Convert timestamps to datetime and then to date for daily aggregation
    df_prices['date'] = pd.to_datetime(df_prices['timestamp'], unit='ms').dt.date
    df_market_caps['date'] = pd.to_datetime(df_market_caps['timestamp'], unit='ms').dt.date
    df_total_volumes['date'] = pd.to_datetime(df_total_volumes['timestamp'], unit='ms').dt.date

    # Group by date and take the last value for simplicity or average, depending on your analysis needs
    # For daily data, the last value of the day is often used.
    df_prices = df_prices.groupby('date')['price'].last().reset_index()
    df_market_caps = df_market_caps.groupby('date')['market_cap'].last().reset_index()
    df_total_volumes = df_total_volumes.groupby('date')['total_volume'].last().reset_index()

    # Merge dataframes
    df = pd.merge(df_prices, df_market_caps, on='date', how='outer')
    df = pd.merge(df, df_total_volumes, on='date', how='outer')

    df['coin_id'] = coin_id
    df['extraction_timestamp'] = datetime.utcnow()

    return df[['coin_id', 'date', 'price', 'market_cap', 'total_volume', 'extraction_timestamp']]

def extract_and_load_coingecko_data(**kwargs):
    """
    Airflow callable function to extract CoinGecko data and load into Snowflake.
    """
    coin_ids = kwargs.get('coin_ids', ['bitcoin', 'ethereum', 'ripple', 'litecoin', 'cardano'])
    vs_currency = kwargs.get('vs_currency', 'usd')

    snowflake_hook = SnowflakeHook(snowflake_conn_id='snowflake_default') # Ensure this matches your Airflow connection ID
    table_name = "COINGECKO_HISTORICAL_PRICES"
    schema_name = "RAW"
    database_name = "CRYPTO_DB"

    all_data_frames = []

    for coin_id in coin_ids:
        print(f"Fetching data for {coin_id}...")
        raw_data = get_market_chart(coin_id, vs_currency=vs_currency)
        df = transform_market_chart_data(raw_data, coin_id)
        if not df.empty:
            all_data_frames.append(df)
            print(f"Successfully processed {len(df)} records for {coin_id}.")
        else:
            print(f"No data processed for {coin_id}.")

    if not all_data_frames:
        print("No data collected across all coins. Exiting.")
        return

    final_df = pd.concat(all_data_frames, ignore_index=True)

    # Convert pandas DataFrame to a list of tuples for Snowflake insertion
    # Ensure column order matches your Snowflake table
    # Convert 'date' column to string for direct insertion, or to datetime if Snowflake handles it directly
    final_df['date'] = final_df['date'].astype(str)

    # Define the Snowflake table structure. This is crucial for consistency.
    # Note: Snowflake typically infers types well, but explicit definition helps.
    columns_to_load = ['coin_id', 'date', 'price', 'market_cap', 'total_volume', 'extraction_timestamp']
    data_to_load = [tuple(row) for row in final_df[columns_to_load].itertuples(index=False)]

    if data_to_load:
        print(f"Loading {len(data_to_load)} records into Snowflake table {database_name}.{schema_name}.{table_name}...")
        # Use a temporary stage for bulk loading, or insert directly if the volume is small.
        # For large data, consider using SnowflakeHook's `copy_into_table` method with a staged file.
        # For simplicity, let's use a direct insert for now.
        # Note: This is an OVERWRITE strategy for demonstration. For production, consider incremental loads or UPSERTs.

        # Create table if not exists (only if you want the hook to manage schema)
        # Or, create the table manually in Snowflake first.
        # Manual table creation is generally preferred for production.
        # CREATE TABLE CRYPTO_DB.RAW.COINGECKO_HISTORICAL_PRICES (
        #     coin_id VARCHAR,
        #     date DATE,
        #     price FLOAT,
        #     market_cap FLOAT,
        #     total_volume FLOAT,
        #     extraction_timestamp TIMESTAMP_LTZ
        # );

        # Load data using pandas_to_snowflake. The hook's run method might be more flexible.
        # Alternatively, use a custom SQL INSERT or COPY INTO from a Pandas DataFrame saved to a temporary file.

        # Let's use the execute method with a list of tuples and manage the table creation/truncation ourselves.
        # This gives more control over the schema and ensures atomicity.
        try:
            with snowflake_hook.get_conn() as conn:
                with conn.cursor() as cur:
                    # Truncate table before inserting (for full refresh)
                    cur.execute(f"TRUNCATE TABLE {database_name}.{schema_name}.{table_name}")

                    # Prepare the insert statement
                    placeholders = ', '.join(['%s'] * len(columns_to_load))
                    column_names = ', '.join(columns_to_load)
                    insert_sql = f"INSERT INTO {database_name}.{schema_name}.{table_name} ({column_names}) VALUES ({placeholders})"

                    # Execute batch insert
                    cur.executemany(insert_sql, data_to_load)
                    conn.commit()
            print(f"Successfully loaded {len(data_to_load)} records to Snowflake.")
        except Exception as e:
            print(f"Error loading data to Snowflake: {e}")
            raise # Re-raise to fail the Airflow task
    else:
        print("No data to load into Snowflake.")

if __name__ == "__main__":
    # Example local run
    # For actual Airflow execution, the function extract_and_load_coingecko_data will be called by PythonOperator
    print("Running local test extraction...")
    # Make sure your Snowflake connection details are set as environment variables for local testing
    # Or mock the SnowflakeHook for isolated testing
    # This block won't be run by Airflow, but useful for testing the script's core logic
    extract_and_load_coingecko_data(
        coin_ids=['bitcoin'],
        vs_currency='usd',
        # You might need to set up a dummy Snowflake connection or mock the hook for local script testing outside Airflow
    )
    print("Local test extraction complete.")

In [ ]:
# ... (your existing functions: get_coin_list, get_market_chart, transform_market_chart_data) ...

def extract_and_load_coingecko_data(**kwargs):
    """
    Airflow callable function to extract CoinGecko data and load into Snowflake.
    """
    coin_ids = kwargs.get('coin_ids', ['bitcoin', 'ethereum', 'ripple', 'litecoin', 'cardano'])
    vs_currency = kwargs.get('vs_currency', 'usd')

    # IMPORTANT: For standalone testing, you'll need to pass Snowflake credentials
    # or ensure they are available as environment variables your script can access.
    # Airflow manages this via Connections, but for direct Python, you need an alternative.
    # The simplest for local testing is often to ensure the SnowflakeHook can pick up
    # env vars (SNOWFLAKE_USER, SNOWFLAKE_PASSWORD, etc.) or hardcode them temporarily (not recommended for Git).

    # For now, let's assume the SnowflakeHook can implicitly connect if you have
    # your account details in a configuration file or env vars where it looks.
    # OR, for pure local testing, you can comment out Snowflake related parts
    # and just focus on data fetching and transformation.

    snowflake_hook = SnowflakeHook(snowflake_conn_id='snowflake_default') # Airflow connection ID
    table_name = "COINGECKO_HISTORICAL_PRICES"
    schema_name = "RAW"
    database_name = "CRYPTO_DB"

    all_data_frames = []

    for coin_id in coin_ids:
        print(f"Fetching data for {coin_id}...")
        raw_data = get_market_chart(coin_id, vs_currency=vs_currency)
        df = transform_market_chart_data(raw_data, coin_id)
        if not df.empty:
            all_data_frames.append(df)
            print(f"Successfully processed {len(df)} records for {coin_id}.")
        else:
            print(f"No data processed for {coin_id}.")

    if not all_data_frames:
        print("No data collected across all coins. Exiting.")
        return

    final_df = pd.concat(all_data_frames, ignore_index=True)

    final_df['date'] = final_df['date'].astype(str)

    columns_to_load = ['coin_id', 'date', 'price', 'market_cap', 'total_volume', 'extraction_timestamp']
    data_to_load = [tuple(row) for row in final_df[columns_to_load].itertuples(index=False)]

    if data_to_load:
        print(f"Attempting to load {len(data_to_load)} records into Snowflake table {database_name}.{schema_name}.{table_name}...")
        try:
            with snowflake_hook.get_conn() as conn:
                with conn.cursor() as cur:
                    cur.execute(f"TRUNCATE TABLE {database_name}.{schema_name}.{table_name}")
                    placeholders = ', '.join(['%s'] * len(columns_to_load))
                    column_names = ', '.join(columns_to_load)
                    insert_sql = f"INSERT INTO {database_name}.{schema_name}.{table_name} ({column_names}) VALUES ({placeholders})"
                    cur.executemany(insert_sql, data_to_load)
                    conn.commit()
            print(f"Successfully loaded {len(data_to_load)} records to Snowflake.")
        except Exception as e:
            print(f"Error loading data to Snowflake: {e}")
            # Do NOT re-raise here if you want the script to continue for other tests
            # raise # For standalone testing, you might not want to crash the script
    else:
        print("No data to load into Snowflake.")


# --- Add this block for standalone testing ---
if __name__ == "__main__":
    print("--- Running extract_coingecko_data.py in standalone test mode ---")
    # You need to provide the 'kwargs' that Airflow would normally pass.
    # For standalone, we can hardcode some values or read from config/env vars.
    test_kwargs = {
        'coin_ids': ['bitcoin', 'ethereum', 'dogecoin'], # Pick a few coins for quick testing
        'vs_currency': 'usd',
    }

    # You might need to set environment variables for Snowflake connection for this to work
    # if SnowflakeHook doesn't find 'snowflake_default' without Airflow running.
    # For simple testing, you can temporarily hardcode credentials here (NOT for production code!)
    # import os
    # os.environ['SNOWFLAKE_USER'] = 'CRYPTO_USER'
    # os.environ['SNOWFLAKE_PASSWORD'] = 'YourStrongPassword!'
    # os.environ['SNOWFLAKE_ACCOUNT'] = '<your_snowflake_account_identifier>'
    # os.environ['SNOWFLAKE_WAREHOUSE'] = 'CRYPTO_WH'
    # os.environ['SNOWFLAKE_DATABASE'] = 'CRYPTO_DB'
    # os.environ['SNOWFLAKE_SCHEMA'] = 'RAW'
    # os.environ['SNOWFLAKE_ROLE'] = 'CRYPTO_ROLE'


    extract_and_load_coingecko_data(**test_kwargs)
    print("--- Standalone test complete ---")